Importing The Dependencies

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Device Configuration

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


Data Collecting & Preprocessing

In [3]:
data = load_breast_cancer()
X = data.data
y = data.target

In [4]:
X, y

(array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
         1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
         8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
         8.758e-02],
        ...,
        [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
         7.820e-02],
        [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
         1.240e-01],
        [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
         7.039e-02]]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
 

Splitting The Data

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [6]:
print(X.shape, X_train.shape, X_test.shape)

(569, 30) (455, 30) (114, 30)


In [7]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
X_train_scaled

array([[-1.44075296, -0.43531947, -1.36208497, ...,  0.9320124 ,
         2.09724217,  1.88645014],
       [ 1.97409619,  1.73302577,  2.09167167, ...,  2.6989469 ,
         1.89116053,  2.49783848],
       [-1.39998202, -1.24962228, -1.34520926, ..., -0.97023893,
         0.59760192,  0.0578942 ],
       ...,
       [ 0.04880192, -0.55500086, -0.06512547, ..., -1.23903365,
        -0.70863864, -1.27145475],
       [-0.03896885,  0.10207345, -0.03137406, ...,  1.05001236,
         0.43432185,  1.21336207],
       [-0.54860557,  0.31327591, -0.60350155, ..., -0.61102866,
        -0.3345212 , -0.84628745]])

In [9]:
X_train = torch.tensor(X_train_scaled, dtype = torch.float32).to(device)
y_train = torch.tensor(y_train, dtype = torch.float32).to(device)
X_test = torch.tensor(X_test_scaled, dtype = torch.float32).to(device)
y_test = torch.tensor(y_test, dtype = torch.float32).to(device)

Building The NN Architecture

In [10]:
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(NeuralNet, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, output_size)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    out = self.sigmoid(out)
    return out

In [11]:
input_size = X_train.shape[1]
hidden_size = 64
output_size = 1
learning_rate = 0.001
no_of_epochs = 100

In [12]:
model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [23]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

Training The Model

In [25]:
for epoch in range(no_of_epochs):
  model.train()
  optimizer.zero_grad()
  outputs = model(X_train)
  loss = criterion(outputs, y_train.view(-1, 1))
  loss.backward()
  optimizer.step()

  with torch.no_grad():
    predicted = outputs.round()
    correct = (predicted == y_train.view(-1, 1)).float().sum()
    accuracy = correct / y_train.size(0)

  if (epoch + 1) % 10 == 0:
    print(f"Epoch [{epoch + 1}/{no_of_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy.item() * 100:.2f}%")


Epoch [10/100], Loss: 0.4718, Accuracy: 91.65%
Epoch [20/100], Loss: 0.3597, Accuracy: 92.97%
Epoch [30/100], Loss: 0.2783, Accuracy: 93.63%
Epoch [40/100], Loss: 0.2216, Accuracy: 94.07%
Epoch [50/100], Loss: 0.1827, Accuracy: 95.16%
Epoch [60/100], Loss: 0.1557, Accuracy: 96.48%
Epoch [70/100], Loss: 0.1362, Accuracy: 96.70%
Epoch [80/100], Loss: 0.1217, Accuracy: 97.14%
Epoch [90/100], Loss: 0.1105, Accuracy: 97.14%
Epoch [100/100], Loss: 0.1016, Accuracy: 98.02%


Model Evaluation

In [26]:
model.eval()
with torch.no_grad():
  outputs = model(X_train)
  predicted = outputs.round()
  correct = (predicted == y_train.view(-1,1)).float().sum()
  accuracy = correct/y_train.size(0)
  print(f"Accuracy on training data: {accuracy.item() * 100:.2f}%")

Accuracy on training data: 98.02%


In [27]:
model.eval()
with torch.no_grad():
  outputs = model(X_test)
  predicted = outputs.round()
  correct = (predicted == y_test.view(-1,1)).float().sum()
  accuracy = correct/y_test.size(0)
  print(f"Accuracy on test data: {accuracy.item() * 100:.2f}%")

Accuracy on test data: 97.37%
